In [1]:
%pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 2.0 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149870 sha256=38dc61c15c7004848e34f96a030bbe4dada9ef776c81710fc4887263708b19bd
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [2]:
import pickle
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Reshape, Dense, Bidirectional, Dropout, LSTM 
,StringLookup, ConvLSTM2D)
from tensorflow.keras import Model
from tensorflow.keras.backend import ctc_decode, ctc_batch_cost
from tensorflow.keras.layers import Layer

import Levenshtein as lev



In [4]:
with open('./labeltensor_shpad.pkl','rb') as ry:
    lab_tensor_sh = pickle.load(ry)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
with open('./drive/MyDrive/imgtensor_sh.pkl','rb') as wabi:
    img_tensor_sh = pickle.load(wabi)

In [35]:
with open('./drive/MyDrive/imgtensor_trsh.pkl','rb') as arbi:
  img_tensor_trsh = pickle.load(arbi)

In [7]:
charlist = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,-+1234567890'

In [8]:
def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    #trying this method here-  
    #https://stackoverflow.com/questions/64321779/how-to-use-tf-ctc-loss-with-variable-length-features-and-labels
    label_length = tf.math.count_nonzero(y_true, axis= -1, keepdims = True)
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return tf.reduce_mean(loss)

In [33]:
def decode_preds(pred_model,start,end,imten):
    preds = pred_model.predict(imten[start:end])
    len_tensor = tf.convert_to_tensor([len([l for l in lab if l>0]) for lab in lab_tensor_sh[start:end]])
    pred_texts = ctc_decode(preds,len_tensor,greedy=True)
    guess_list =  [''.join([charlist[p-1] for p in pred.numpy() if p > -1]) for pred in pred_texts[0][0]]
    prob_list = pred_texts[1].numpy()
    return guess_list,prob_list

In [29]:
def post_distance(pred_model,start,end):
    preds, _ = decode_preds(pred_model,start,end)
    trues = [''.join([charlist[p-1] for p in lab.numpy() if p>0]) for lab in lab_tensor_sh[start:end]]
    return np.mean([lev.distance(trues[i],preds[i]) for i in range(end-start)])

In [11]:
input_img = Input(shape=(480,679,1),name='image',dtype="float32")



x = Conv2D(32,(3,3),activation='relu',name='Conv1')(input_img)
x = MaxPooling2D((2,2), name = "pool1")(x)
x = Dropout(0.3,name = 'drop1')(x)


x = Conv2D(64,(3,3),activation='relu',name='Conv2')(x)
x = MaxPooling2D((2,2), name = "pool2")(x)
x = Dropout(0.3, name = 'drop2')(x)

x = Conv2D(128,(4,4), activation= 'relu',name='Conv3')(x)
x = Conv2D(128,(5,5),activation='relu', name = 'Conv4')(x)
x = MaxPooling2D((2,2), name = 'pool3')(x)
x = Dropout(0.3,name = 'drop3')(x)

x = Conv2D(128,(5,5), activation= 'relu', name = 'Conv5')(x)

shape = (76,6528)

x = Reshape(target_shape=shape, name = 'reshape')(x)
x = Dense(64, activation='relu', name ='dense1')(x)
x = Dropout(0.2,name = 'drop4')(x)

x = Bidirectional(LSTM(256,return_sequences=True, dropout=0.25), name = 'LSTM1')(x)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.25), name = 'LSTM2')(x)
x = Bidirectional(LSTM(64,return_sequences=True,dropout=0.25), name = 'LSTM3')(x)



output = Dense(len(charlist)+1,activation='softmax',name='dense2')(x)
tf.keras.layers.Softmax()
tf.keras.layers.Masking()

model = Model(input_img,output,name='basic-OCRplusvar')
model.compile(optimizer='adam',loss=CTCLoss)

In [12]:
model.summary()

Model: "basic-OCRplusvar"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 480, 679, 1)]     0         
                                                                 
 Conv1 (Conv2D)              (None, 478, 677, 32)      320       
                                                                 
 pool1 (MaxPooling2D)        (None, 239, 338, 32)      0         
                                                                 
 drop1 (Dropout)             (None, 239, 338, 32)      0         
                                                                 
 Conv2 (Conv2D)              (None, 237, 336, 64)      18496     
                                                                 
 pool2 (MaxPooling2D)        (None, 118, 168, 64)      0         
                                                                 
 drop2 (Dropout)             (None, 118, 168, 64) 

In [13]:
eps = 100
history = model.fit(img_tensor_sh,lab_tensor_sh,validation_split=0.2, epochs=eps)

Epoch 1/100
36/36 [==============================] - 52s 678ms/step - loss: 49.8297 - val_loss: 35.9245
Epoch 2/100
36/36 [==============================] - 19s 533ms/step - loss: 32.3063 - val_loss: 35.0056
Epoch 3/100
36/36 [==============================] - 19s 535ms/step - loss: 31.9972 - val_loss: 34.7387
Epoch 4/100
36/36 [==============================] - 19s 539ms/step - loss: 31.7857 - val_loss: 34.7194
Epoch 5/100
36/36 [==============================] - 19s 540ms/step - loss: 31.5149 - val_loss: 34.5139
Epoch 6/100
36/36 [==============================] - 19s 542ms/step - loss: 31.2564 - val_loss: 34.0513
Epoch 7/100
36/36 [==============================] - 20s 544ms/step - loss: 31.0572 - val_loss: 34.1285
Epoch 8/100
36/36 [==============================] - 20s 545ms/step - loss: 30.9250 - val_loss: 33.9341
Epoch 9/100
36/36 [==============================] - 20s 547ms/step - loss: 30.8538 - val_loss: 33.7224
Epoch 10/100
36/36 [==============================] - 20s 549ms/

In [14]:
eps = 5
history = model.fit(img_tensor_sh,lab_tensor_sh,validation_split=0.2, epochs=eps)

Epoch 1/5
36/36 [==============================] - 20s 543ms/step - loss: 15.9905 - val_loss: 26.8490
Epoch 2/5
36/36 [==============================] - 19s 540ms/step - loss: 16.0485 - val_loss: 24.7751
Epoch 3/5
36/36 [==============================] - 19s 539ms/step - loss: 15.7771 - val_loss: 25.2508
Epoch 4/5
36/36 [==============================] - 19s 539ms/step - loss: 15.4970 - val_loss: 24.2212
Epoch 5/5
36/36 [==============================] - 19s 539ms/step - loss: 15.3173 - val_loss: 24.8624


In [15]:
model.save('./ocrplusvar105')

INFO:tensorflow:Assets written to: ./ocrplusvar105/assets


INFO:tensorflow:Assets written to: ./ocrplusvar105/assets


In [16]:
!zip -r  ./ocrplusvar105.zip ./ocrplusvar105/

  adding: ocrplusvar105/ (stored 0%)
  adding: ocrplusvar105/variables/ (stored 0%)
  adding: ocrplusvar105/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: ocrplusvar105/variables/variables.index (deflated 73%)
  adding: ocrplusvar105/assets/ (stored 0%)
  adding: ocrplusvar105/keras_metadata.pb (deflated 94%)
  adding: ocrplusvar105/saved_model.pb (deflated 90%)


In [17]:
prediction_model = Model(model.get_layer(name='image').input, model.get_layer(name='dense2').output)

In [20]:
guesses, probs = decode_preds(prediction_model,0,1200)

In [21]:
guesses[0]

'M'

In [30]:
post_distance(prediction_model,0,1407)

7.240938166311301

In [32]:
[guess for guess in guesses if len(guess)>1]

['---',
 '---',
 '---',
 '---',
 '---',
 'So',
 '---',
 '---',
 '27',
 '---',
 '--',
 '---',
 '--',
 'US',
 '---',
 '---',
 '---',
 '---',
 '---',
 '--',
 '---',
 '---',
 '---',
 '2T',
 '---',
 '---',
 '---',
 '---',
 '---',
 '---',
 '---',
 '---',
 '---',
 '---',
 '---',
 'US',
 '---',
 '---',
 '---',
 '---',
 '---',
 '--',
 'VT',
 '---',
 '---',
 '--',
 '--',
 '27',
 '---',
 '---',
 '---',
 '---',
 '---',
 'VT',
 '---',
 '---',
 '---',
 'VT',
 '---',
 '---',
 '--',
 '---',
 '---',
 'Hh',
 '---',
 '---',
 '---',
 '---',
 '---',
 'US',
 '--',
 '---',
 'Lo',
 '---',
 '---',
 'pr',
 '2S',
 '26',
 '29',
 '---',
 '---',
 '--',
 '2S',
 'VT',
 '---',
 '---',
 '--',
 '---',
 '--',
 '--',
 '---',
 '---',
 'So',
 '---',
 'So',
 '---',
 '---',
 'Al',
 '2T',
 'So',
 '---',
 '--',
 '---',
 '--',
 '--',
 'Amol ',
 '---',
 '---',
 'M9',
 '---',
 '--',
 'Oh',
 '2S',
 '---',
 '---',
 '---',
 '--',
 '---',
 '---',
 '2S',
 '---',
 'VT',
 '--',
 'So',
 '27',
 '---',
 'VT',
 '---',
 '---',
 '17',
 '---',


In [28]:
[''.join([charlist[p-1] for p in lab.numpy() if p>0]) for lab in lab_tensor_sh[0:20]]

['Secretary',
 'Neg',
 '--- Steven A',
 '36',
 'M',
 'Flowers Ruby J',
 '---, Frank H',
 '---, Vera P',
 'Edwards  Ada',
 'Welch Ernestine',
 'Davis Henry A',
 'Alabama',
 'W',
 'Head',
 'Steward Mildred R',
 '69',
 'M',
 'H',
 'No one at home',
 'Greece']

In [38]:
input_img = Input(shape=(679,480,1),name='image',dtype="float32")



x = Conv2D(32,(3,3),activation='relu',name='Conv1')(input_img)
x = MaxPooling2D((2,2), name = "pool1")(x)
x = Dropout(0.3,name = 'drop1')(x)


x = Conv2D(64,(3,3),activation='relu',name='Conv2')(x)
x = MaxPooling2D((2,2), name = "pool2")(x)
x = Dropout(0.3, name = 'drop2')(x)

x = Conv2D(128,(4,4), activation= 'relu',name='Conv3')(x)
x = Conv2D(128,(5,5),activation='relu', name = 'Conv4')(x)
x = MaxPooling2D((2,2), name = 'pool3')(x)
x = Dropout(0.3,name = 'drop3')(x)

x = Conv2D(128,(5,5), activation= 'relu', name = 'Conv5')(x)

shape = (76,6528)

x = Reshape(target_shape=shape, name = 'reshape')(x)
x = Dense(64, activation='relu', name ='dense1')(x)
x = Dropout(0.2,name = 'drop4')(x)

x = Bidirectional(LSTM(256,return_sequences=True, dropout=0.25), name = 'LSTM1')(x)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.25), name = 'LSTM2')(x)
x = Bidirectional(LSTM(64,return_sequences=True,dropout=0.25), name = 'LSTM3')(x)



output = Dense(len(charlist)+1,activation='softmax',name='dense2')(x)

model2 = Model(input_img,output,name='basic-OCRplusvartpose')
model2.compile(optimizer='adam',loss=CTCLoss)

In [39]:
model2.summary()

Model: "basic-OCRplusvartpose"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 679, 480, 1)]     0         
                                                                 
 Conv1 (Conv2D)              (None, 677, 478, 32)      320       
                                                                 
 pool1 (MaxPooling2D)        (None, 338, 239, 32)      0         
                                                                 
 drop1 (Dropout)             (None, 338, 239, 32)      0         
                                                                 
 Conv2 (Conv2D)              (None, 336, 237, 64)      18496     
                                                                 
 pool2 (MaxPooling2D)        (None, 168, 118, 64)      0         
                                                                 
 drop2 (Dropout)             (None, 168, 118,

In [40]:
eps = 100
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)
history = model2.fit(img_tensor_trsh,lab_tensor_sh,validation_split=0.2, epochs=eps,callbacks=[early_stopping])

Epoch 1/100
36/36 [==============================] - 42s 785ms/step - loss: 54.1772 - val_loss: 35.7461
Epoch 2/100
36/36 [==============================] - 21s 578ms/step - loss: 32.3469 - val_loss: 35.2852
Epoch 3/100
36/36 [==============================] - 21s 581ms/step - loss: 32.0104 - val_loss: 34.9697
Epoch 4/100
36/36 [==============================] - 21s 583ms/step - loss: 31.7497 - val_loss: 34.4743
Epoch 5/100
36/36 [==============================] - 21s 587ms/step - loss: 31.4779 - val_loss: 34.0926
Epoch 6/100
36/36 [==============================] - 21s 589ms/step - loss: 31.3077 - val_loss: 34.0385
Epoch 7/100
36/36 [==============================] - 21s 592ms/step - loss: 31.1348 - val_loss: 33.9522
Epoch 8/100
36/36 [==============================] - 21s 593ms/step - loss: 31.0199 - val_loss: 33.7543
Epoch 9/100
36/36 [==============================] - 21s 595ms/step - loss: 30.3959 - val_loss: 32.1383
Epoch 10/100
36/36 [==============================] - 21s 596ms/

In [45]:
preds = Model(model2.get_layer(name="image").input, model.get_layer(name='dense2').output)

ValueError: ignored

In [43]:
model2.save('./ocrplusvartpose69')

INFO:tensorflow:Assets written to: ./ocrplusvartpose69/assets


INFO:tensorflow:Assets written to: ./ocrplusvartpose69/assets


In [44]:
!zip -r  ./ocrplusvartpose69.zip ./ocrplusvartpose69/

  adding: ocrplusvartpose69/ (stored 0%)
  adding: ocrplusvartpose69/variables/ (stored 0%)
  adding: ocrplusvartpose69/variables/variables.data-00000-of-00001 (deflated 9%)
  adding: ocrplusvartpose69/variables/variables.index (deflated 73%)
  adding: ocrplusvartpose69/assets/ (stored 0%)
  adding: ocrplusvartpose69/keras_metadata.pb (deflated 94%)
  adding: ocrplusvartpose69/saved_model.pb (deflated 90%)
